<a href="https://colab.research.google.com/github/rrfsantos/BNDES-premiodadosabertos/blob/main/Q_Learning_FrozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q* Learning with FrozenLake 4x4 

Neste Notebook, implementaremos um agente <b> que reproduz FrozenLake. </b>

![alt text](http://simoninithomas.com/drlc/Qlearning/frozenlake4x4.png)

O objetivo deste jogo é <b> ir do estado inicial (S) para o estado objetivo (G) </b> caminhando apenas em ladrilhos congelados (F) e evitando buracos (H). No entanto, o gelo é escorregadio, ** então você nem sempre se moverá na direção que pretende (ambiente estocástico) **

By [lukewys](https://github.com/lukewys)

## Prerequisites 🏗️
Before diving on the notebook **you need to understand**:
- The foundations of Reinforcement learning (MC, TD, Rewards hypothesis...) [Article](https://medium.freecodecamp.org/an-introduction-to-reinforcement-learning-4339519de419)
- Q-learning [Article](https://medium.freecodecamp.org/diving-deeper-into-reinforcement-learning-with-q-learning-c18d0db58efe)
- In the [video version](https://www.youtube.com/watch?v=q2ZOEFAaaI0)  we implemented a Q-learning agent that learns to play OpenAI Taxi-v2 🚕 with Numpy.

## Step -1: Instale as dependências no Google Colab

In [ ]:
#!pip install numpy
!pip install gym

## Step 0: Importar as dependências 📚
Usamos 3 bibliotecas:
- `Numpy` para nosso Qtable
- `OpenAI Gym` para nosso ambiente FrozenLake
- `Random` para gerar números aleatórios

In [ ]:
import numpy as np
import gym
import random

## Step 1: Crie o ambiente 🎮
- Aqui vamos criar o ambiente FrozenLake 8x8.
- OpenAI Gym é uma biblioteca <b> composta por vários ambientes que podemos usar para treinar nossos agentes. </b>
- No nosso caso optamos por usar Frozen Lake.

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/amidar.bin
copying assault.bin from ROMS/Assau

In [ ]:
env = gym.make("FrozenLake-v0")

## Step 2: Crie a tabela Q e inicialize-a 🗄️
- Agora, vamos criar nossa tabela Q, para saber de quantas linhas (estados) e colunas (ações) precisamos, precisamos calcular o action_size e o state_size
- OpenAI Gym nos fornece uma maneira de fazer isso: `env.action_space.n` e` env.observation_space.n`

In [ ]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [ ]:
state_size

16

In [ ]:
# Crie nossa tabela Q com linhas state_size e colunas action_size (16x4)
print('state_size ',state_size) # 16
print('action_size ',action_size) # 4
qtable = np.zeros((state_size, action_size))
print(qtable)

state_size  16
action_size  4
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Step 3: Crie os hiperparâmetros ⚙️
- Aqui, vamos especificar os hiperparâmetros

In [ ]:
total_episodes = 30000       # Total episodes or epochs (i é o índice de iteração)
learning_rate = 0.35         # Learning rate (α é a taxa de aprendizado)
max_steps = 99               # Max steps per episode or actions
gamma = 0.99                 # Discounting rate (γ é a taxa de desconto para recompensas futuras)

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start (Inicia com muitas ações aleatórias)
min_epsilon = 0.01            # Minimum exploration probability (Depois faz exploitation)
decay_rate = 0.005            # Exponential decay rate for exploration prob 

## Step 4: O algoritmo de aprendizagem Q 🧠
- Agora implementamos o algoritmo de aprendizagem Q:
  ![alt text](http://simoninithomas.com/drlc/Qlearning//qtable_algo.png)


In [ ]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            #print(exp_exp_tradeoff, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)
            
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)
    

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

Score over time: 0.6248333333333334
[[0.52509347 0.38106967 0.39156748 0.38970822]
 [0.23199527 0.18464901 0.23042823 0.36521961]
 [0.32951697 0.31227183 0.31886033 0.34305234]
 [0.0869629  0.30445376 0.2377188  0.34167066]
 [0.56450635 0.2450641  0.39793344 0.33154805]
 [0.         0.         0.         0.        ]
 [0.35349825 0.0257803  0.06701312 0.05721928]
 [0.         0.         0.         0.        ]
 [0.25970436 0.45372543 0.2188027  0.65115544]
 [0.31112783 0.68806472 0.39223019 0.43513889]
 [0.78036889 0.2625356  0.37731428 0.18885764]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.42445022 0.43409033 0.76221266 0.33310905]
 [0.58943046 0.92307609 0.59127712 0.57135855]
 [0.         0.         0.         0.        ]]


## Step 5: Use nossa tabela Q para jogar FrozenLake! 👾
- Depois de 10.000 episódios, nossa tabela-Q pode ser usada como uma "folha de cheats" para jogar FrozenLake "
- Ao rodar este celular você pode ver nosso agente jogando FrozenLake.

In [ ]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 15:
                print("We reached our Goal 🏆")
            else:
                print("We fell into a hole ☠️")
            
            # We print the number of step it took.
            print("Number of steps", step)
            
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 46
****************************************************
EPISODE  1
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 63
****************************************************
EPISODE  2
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 39
****************************************************
EPISODE  3
  (Left)
SFFF
FHFH
FFFH
HFFG
We fell into a hole ☠️
Number of steps 10
****************************************************
EPISODE  4
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 12
